In [ ]:
import oml
import xgboost as xgb
import numpy as np
import pandas as pd


Connect to database

In [ ]:
oml.connect("soe","soe",dsn='(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(service_name=app_root)))')

Load data from files to database tables

In [ ]:
from sklearn.datasets import load_svmlight_file

dtrain = xgb.DMatrix('agaricus.txt.train')
col_names=dtrain.feature_names

train_data = load_svmlight_file('agaricus.txt.train')

train_X = train_data[0].toarray()
train_y = train_data[1]
train_df=pd.DataFrame(data=train_X,columns=col_names[1:127])
train_target_df=pd.DataFrame(data=train_y)

oml.create(train_df,table='TRAIN_DF')
oml.create(train_target_df,table='TARGET_DF')

In [ ]:
test_data = load_svmlight_file('agaricus.txt.test')

test_X = test_data[0].toarray()
test_y = test_data[1]
test_df=pd.DataFrame(data=test_X,columns=col_names[1:127])
test_target_df=pd.DataFrame(data=test_y)

oml.create(test_df,table='TEST_DF')
oml.create(test_target_df,table='TEST_TARGET_DF')

Create functions to train and use the model. Store them in the script repository

In [ ]:
def train_xgboost():
    import oml
    import xgboost as xgb
    
    dat=oml.sync(table="TRAIN_DF").pull()
    target=oml.sync(table="TARGET_DF").pull()
    
    dtrain=xgb.DMatrix(dat, label=target)
    param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
    num_round = 2
    bst = xgb.train(param, dtrain, num_round)
    mod=bst.save_raw()
    oml.ds.save(objs={'xgboost_raw':mod}, name="MED", append=True)
    return mod

oml.script.create("train_xgboost", train_xgboost, overwrite = True)

In [ ]:
def use_xgboost():
    import oml
    import xgboost as xgb
    
    objs=oml.ds.load(name="MED", to_globals=False)
    bst2 = xgb.Booster({'nthread': '4'})
    bst2.load_model(objs['xgboost_raw'])
    
    dat=oml.sync(table="TEST_DF").pull()
    target=oml.sync(table="TEST_TARGET_DF").pull()
    
    dtest=xgb.DMatrix(dat, label=target)
    pred = bst2.predict(dtest)
    return pred

oml.script.create("use_xgboost", use_xgboost, overwrite = True)

In [ ]:
oml.script.dir(name="xgboost$",regex_match=True)

Run embedded execution for the xgboost training

In [ ]:
res = oml.do_eval(func='train_xgboost',oml_connect=True)

Look for the model in database's data store

In [ ]:
oml.ds.dir(name="MED")

In [ ]:
oml.ds.describe(name="MED")

Use xgboost model with test data

In [ ]:
res = oml.do_eval(func='use_xgboost',oml_connect=True)
res

Create function to use the model with parameters

In [ ]:
def use_xgboost_pars(f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126):
    import oml
    import pandas as pd
    import xgboost as xgb
    
    objs=oml.ds.load(name="MED", to_globals=False)
    bst2 = xgb.Booster({'nthread': '4'})
    bst2.load_model(objs['xgboost_raw'])
    
    dat=pd.DataFrame({'f1': f1, 'f2': f2, 'f3': f3, 'f4': f4, 'f5': f5, 'f6': f6, 'f7': f7, 'f8': f8, 'f9': f9, 'f10': f10, 'f11': f11, 'f12': f12, 'f13': f13, 'f14': f14, 'f15': f15, 'f16': f16, 'f17': f17, 'f18': f18, 'f19': f19, 'f20': f20, 'f21': f21, 'f22': f22, 'f23': f23, 'f24': f24, 'f25': f25, 'f26': f26, 'f27': f27, 'f28': f28, 'f29': f29, 'f30': f30, 'f31': f31, 'f32': f32, 'f33': f33, 'f34': f34, 'f35': f35, 'f36': f36, 'f37': f37, 'f38': f38, 'f39': f39, 'f40': f40, 'f41': f41, 'f42': f42, 'f43': f43, 'f44': f44, 'f45': f45, 'f46': f46, 'f47': f47, 'f48': f48, 'f49': f49, 'f50': f50, 'f51': f51, 'f52': f52, 'f53': f53, 'f54': f54, 'f55': f55, 'f56': f56, 'f57': f57, 'f58': f58, 'f59': f59, 'f60': f60, 'f61': f61, 'f62': f62, 'f63': f63, 'f64': f64, 'f65': f65, 'f66': f66, 'f67': f67, 'f68': f68, 'f69': f69, 'f70': f70, 'f71': f71, 'f72': f72, 'f73': f73, 'f74': f74, 'f75': f75, 'f76': f76, 'f77': f77, 'f78': f78, 'f79': f79, 'f80': f80, 'f81': f81, 'f82': f82, 'f83': f83, 'f84': f84, 'f85': f85, 'f86': f86, 'f87': f87, 'f88': f88, 'f89': f89, 'f90': f90, 'f91': f91, 'f92': f92, 'f93': f93, 'f94': f94, 'f95': f95, 'f96': f96, 'f97': f97, 'f98': f98, 'f99': f99, 'f100': f100, 'f101': f101, 'f102': f102, 'f103': f103, 'f104': f104, 'f105': f105, 'f106': f106, 'f107': f107, 'f108': f108, 'f109': f109, 'f110': f110, 'f111': f111, 'f112': f112, 'f113': f113, 'f114': f114, 'f115': f115, 'f116': f116, 'f117': f117, 'f118': f118, 'f119': f119, 'f120': f120, 'f121': f121, 'f122': f122, 'f123': f123, 'f124': f124, 'f125': f125, 'f126': f126}, index=[0])
    target=pd.DataFrame({'0':0}, index=[0])
    
    dtest=xgb.DMatrix(dat, label=target)
    pred = bst2.predict(dtest)

    return pred

In [ ]:
oml.script.create("use_xgboost_pars", use_xgboost_pars, overwrite = True)

Now from sql:

```
SELECT *
FROM table(pyqEval(
'{"f1":0,"f2":0,"f3":0,"f4":1,"f5":0,"f6":0,"f7":0,"f8":0,"f9":1,"f10":0,"f11":1,"f12":0,"f13":0,"f14":0,"f15":0,"f16":0,"f17":0,"f18":0,"f19":0,"f20":0,"f21":1,"f22":0,"f23":0,"f24":0,"f25":0,"f26":0,"f27":0,"f28":0,"f29":0,"f30":1,"f31":0,"f32":0,"f33":0,"f34":1,"f35":0,"f36":1,"f37":0,"f38":0,"f39":0,"f40":1,"f41":0,"f42":0,"f43":0,"f44":0,"f45":0,"f46":0,"f47":0,"f48":0,"f49":0,"f50":0,"f51":1,"f52":0,"f53":1,"f54":0,"f55":0,"f56":0,"f57":0,"f58":1,"f59":0,"f60":0,"f61":0,"f62":0,"f63":0,"f64":0,"f65":1,"f66":0,"f67":0,"f68":0,"f69":1,"f70":0,"f71":0,"f72":0,"f73":0,"f74":0,"f75":0,"f76":0,"f77":1,"f78":0,"f79":0,"f80":0,"f81":0,"f82":0,"f83":0,"f84":0,"f85":0,"f86":1,"f87":0,"f88":1,"f89":0,"f90":0,"f91":0,"f92":1,"f93":0,"f94":0,"f95":1,"f96":0,"f97":0,"f98":0,"f99":0,"f100":0,"f101":0,"f102":1,"f103":0,"f104":0,"f105":1,"f106":0,"f107":0,"f108":0,"f109":0,"f110":0,"f111":0,"f112":0,"f113":0,"f114":0,"f115":0,"f116":0,"f117":1,"f118":0,"f119":0,"f120":0,"f121":0,"f122":0,"f123":0,"f124":1,"f125":0,"f126":0,"oml_connect":1}', 
'{"0":"number"}',
'use_xgboost_pars'));
``` 